In [ ]:
import numpy as np
import pandas as pd 
from catboost import CatBoostClassifier
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
train = pd.read_csv("../input/petfinder-adoption-prediction/train/train.csv")
train.head()

In [ ]:
test = pd.read_csv("../input/petfinder-adoption-prediction/test/test.csv")
test.head()

In [ ]:
sample_submission = pd.read_csv("../input/petfinder-adoption-prediction/test/sample_submission.csv")
sample_submission.head()

In [ ]:
categorical_columns = ["Type", "Breed1", "Breed2", "Gender", "Color1", "Color2", "Color3", "MaturitySize", "Vaccinated", "Dewormed", "Sterilized", "Health", "State"]
numerical_columns = ["Age", "Fee", "VideoAmt", "PhotoAmt"]

## Distributions

In [ ]:
for i in range(len(categorical_columns)):
    sns.countplot(x=categorical_columns[i], data=train)
    plt.title("Distribtion of %s"%(categorical_columns[i]))
    plt.show()
for i in range(len(numerical_columns)):
    train[numerical_columns[i]].hist()
    plt.title("Distribtion of %s"%(numerical_columns[i]))
    plt.show()

## Preprocessing

In [ ]:
data = pd.DataFrame()
data = data.append(train)
data = data.append(test)
dfs = []
for i in range(len(categorical_columns)):
    df = pd.get_dummies(data[categorical_columns[i]])
    df.columns = [categorical_columns[i] + "_" + str(column) for column in df.columns]
    dfs.append(df)
numeric_df = data[numerical_columns]
numeric_df = (numeric_df - numeric_df.mean()) / numeric_df.std()
dfs.append(numeric_df)
df = pd.concat(dfs, axis=1)
train_features = df.iloc[:len(train)]
test_features = df.iloc[len(train):]

## Train Validation Split

In [ ]:
from sklearn.model_selection import train_test_split
train_targets = train["AdoptionSpeed"]
X_train, X_val, y_trian, y_val = train_test_split(train_features, train_targets, test_size=0.1)
X_train.shape, X_val.shape, y_trian.shape, y_val.shape

## Modeling

In [ ]:
cat_params = {
    "iterations": 3142,
    "learning_rate": 0.03,
    "depth": 7,
    "verbose": 1000,
    'od_wait': 1000,
    'eval_metric': 'Accuracy',
}
model = CatBoostClassifier(**cat_params)
model.fit(X_train, y_trian, eval_set=(X_val, y_val))

## Submission

In [ ]:
y_pred = model.predict(test_features)
y_pred.shape

In [ ]:
sample_submission["AdoptionSpeed"] = y_pred
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head()